### Convolutional Neural Network

This is a basic convolutional approach to sequence prediction using convolutions through PyTorch!

In [ ]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os, sys
import pickle

In [ ]:
# load data through the data preprocessor
sys.path.append(os.path.abspath('..'))  # add parent directory to sys.path
from data_cleanup import DataProcessor

processor = DataProcessor(input_steps=24, output_steps=1) # get 24 hour sequences, and a 1 hour output
Train, Val, Test = processor.load_and_process_data()

X_train, y_train = Train
X_val, y_val = Val
X_test, y_test = Test

print(f"Train: \n \t X: {X_train.shape}, y: {y_train.shape}")
print(f"Val: \n \t X: {X_val.shape}, y: {y_val.shape}")
print(f"Test: \n \t X: {X_test.shape}, y: {y_test.shape}")

In [ ]:
print(X_train[0].shape)
print(f"Sample tuple\n {X_train[0]}")

In [ ]:
# Convolutional Neural Network: 
# The input will be a sequence of 24 hours of data, with 8 features/channels
# Slide a window through the hours and create data for the next layer 

class CNN(nn.Module): 
    def __init__(self):
        super(CNN, self).__init__() # Inherit from nn.Module
        # Define layers: 3 convolutions (1D) and 2 fully connected layers (very standard)
        self.conv1 = nn.Conv1d(in_channels=8, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=20, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2) # reusable pool layer
        self.fc1 = nn.Linear(in_features=(20), out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=1)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# train
epochs = 10

net = CNN()
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


# turn data into torch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
train_dataset = torch.utils.data.TensorDataset(X_train_t, y_train_t)

# use DataLoader, an iterable object from pytorch, that helps with batch training
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=4, shuffle=True)

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
        # the inputs are given like (batch, time, features),
        # but the model expects (batch, features, time), permutate it: 
        inputs = inputs.permute(0, 2, 1)  

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
# now validate: 
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32)
test_dataset = torch.utils.data.TensorDataset(X_test_t, y_test_t)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

with torch.no_grad():
    test_loss = 0
    for data, label in test_loader:
        data = data.permute(0, 2, 1)
        y_pred = net(data)
        test_loss += F.mse_loss(y_pred, label)

print(f"Test Loss: {test_loss.item()}")


hmmmmmm overfitting ...

#### Pickle 

Saving the model to pickle. This makes it easy to load the model later, if you guys want to test it out, or have your model being tested by somebody else! Pickle is a library that lets you store python objects in a file 

In [ ]:
with open('net.pkl', 'wb') as f:
    pickle.dump(net, f)

In [ ]:
# to load it in the notebook
with open('net.pkl', 'rb') as file:
    net = pickle.load(file)

# that's it now use 'net' as you would normally 